In [1]:
import pandas as pd
import numpy as np
import os
import time
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import Birch
from sklearn.model_selection import train_test_split

In [2]:

# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_01/train_01/',
    './Documents/Research/can-train-and-test/set_02/train_01/',
    './Documents/Research/can-train-and-test/set_03/train_01/',
    './Documents/Research/can-train-and-test/set_04/train_01/'
]

# Test files for DoS attack
dos_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/DoS-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/DoS-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/DoS-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/DoS-2.csv'
]

# Test files for Force neutral attack
fn_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/force-neutral-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/force-neutral-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/force-neutral-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/force-neutral-2.csv'
]

# Test files for RPM attack
rpm_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-2.csv'
]

# Test files for standstill attack
standstill_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/standstill-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/standstill-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/standstill-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/standstill-2.csv'
]

In [3]:
# Function to Load Data
def load_data_from_directory(directory_path):
    data_frames = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            df = pd.read_csv(file_path)
            data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

# Function to Preprocess Data
def hex_to_int(x):
    try:
        return int(str(x), 16)
    except ValueError:
        return 0  # Handle non-hexadecimal values
        
def preprocess_data(df):
    df['timestamp'] = pd.to_datetime(df['timestamp']).astype(np.int64) // 10**9
    df['arbitration_id'] = df['arbitration_id'].apply(hex_to_int)
    df['data_field'] = df['data_field'].apply(hex_to_int)
    return df

# Function to Extract Features and Labels
def extract_features_labels(df, label_col='attack'):
    X = df.drop(columns=label_col)
    y = df[label_col]
    return X, y

# Function to load and combine attack files
def load_combine_attack_files(files):
    data_frames = []
    for file in files:
        df = pd.read_csv(file)
        data_frames.append(df)
    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

In [4]:
# Function to Build and Train BIRCH Model
def train_birch(X_train, y_train):
    # Standardize the Features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Define and Train the BIRCH Model
    birch_model = Birch(threshold=0.5, n_clusters=None)

    # Record training start time
    start_time = time.time()
    
    birch_model.fit(X_train_scaled)
    
    # Calculate training time
    training_time = time.time() - start_time
    
    # Predict clusters on the training data
    train_clusters = birch_model.predict(X_train_scaled)

    # Map clusters to the most frequent attack label in each cluster
    cluster_to_label = {}
    for cluster in np.unique(train_clusters):
        mask = (train_clusters == cluster)
        cluster_to_label[cluster] = np.bincount(y_train[mask]).argmax()

    # Assign predicted labels based on clusters
    train_cluster_labels = np.array([cluster_to_label[cluster] for cluster in train_clusters])

    # Save the model, scaler, and cluster-to-label mapping
    with open('birch_model.pkl', 'wb') as model_file:
        pickle.dump((birch_model, scaler, cluster_to_label), model_file)

    return train_cluster_labels, training_time


In [5]:
# Function to Test BIRCH Model
def test_birch(X_test, y_test):
    # Load the trained model, scaler, and cluster-to-label mapping
    with open('birch_model.pkl', 'rb') as model_file:
        birch_model, scaler, cluster_to_label = pickle.load(model_file)

    # Standardize the Features
    X_test_scaled = scaler.transform(X_test)

    # Record testing start time
    start_time = time.time()
    
    # Predict clusters on the test data
    test_clusters = birch_model.predict(X_test_scaled)

    # Calculate testing time
    testing_time = time.time() - start_time
    
    # Assign predicted labels based on clusters
    test_cluster_labels = np.array([cluster_to_label.get(cluster, -1) for cluster in test_clusters])

    # Evaluate Test Performance
    conf_matrix_test = confusion_matrix(y_test, test_cluster_labels)
    mcc_test = matthews_corrcoef(y_test, test_cluster_labels)
    accuracy_test = accuracy_score(y_test, test_cluster_labels)
    precision_test = precision_score(y_test, test_cluster_labels, average='weighted', zero_division=0)
    recall_test = recall_score(y_test, test_cluster_labels, average='weighted', zero_division=0)
    f1_test = f1_score(y_test, test_cluster_labels, average='weighted', zero_division=0)
    informedness_test = recall_test - (1 - recall_test)
    markedness_test = precision_test - (1 - precision_test)

    return {
        'conf_matrix': conf_matrix_test,
        'mcc': mcc_test,
        'accuracy': accuracy_test,
        'precision': precision_test,
        'recall': recall_test,
        'f1_score': f1_test,
        'informedness': informedness_test,
        'markedness': markedness_test,
        'testing_time': testing_time
    }


In [6]:
# Load and preprocess training data
df_train = pd.concat([load_data_from_directory(train_dir) for train_dir in train_directories], ignore_index=True)
df_train = preprocess_data(df_train)
X_train, y_train = extract_features_labels(df_train)

In [7]:
# Train the BIRCH Model
train_cluster_labels, training_time = train_birch(X_train, y_train)

In [8]:
# Load and preprocess testing data - DOS attack
df_test = pd.concat([pd.read_csv(file) for file in dos_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (DOS Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (DOS Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 6.04 seconds

-----------------------------------------------------------------------
Accuracy: 94.88%
Precision: 90.05%
Recall: 94.88%
F1-Score: 92.40%
Matthews Correlation Coefficient: -0.0031
Informedness: 0.8976
Markedness: 0.8011


In [9]:
# Load and preprocess testing data - Force Neutral
df_test = pd.concat([pd.read_csv(file) for file in fn_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Force neutral Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Force neutral Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 11.25 seconds

-----------------------------------------------------------------------
Accuracy: 99.89%
Precision: 99.82%
Recall: 99.89%
F1-Score: 99.85%
Matthews Correlation Coefficient: -0.0005
Informedness: 0.9977
Markedness: 0.9965


In [10]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in rpm_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (RPM Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (RPM Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 10.37 seconds

-----------------------------------------------------------------------
Accuracy: 99.86%
Precision: 99.76%
Recall: 99.86%
F1-Score: 99.81%
Matthews Correlation Coefficient: -0.0005
Informedness: 0.9972
Markedness: 0.9953


In [11]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in standstill_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Standstill Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Standstill Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 7.56 seconds

-----------------------------------------------------------------------
Accuracy: 99.74%
Precision: 99.54%
Recall: 99.74%
F1-Score: 99.64%
Matthews Correlation Coefficient: -0.0009
Informedness: 0.9948
Markedness: 0.9908


In [12]:
# Test files for DoS attack
double_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/double-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/double-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/double-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/double-2.csv'
]

# Test files for Force neutral attack
triple_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/triple-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/triple-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/triple-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/triple-2.csv'
]

fuzzing_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/fuzzing-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/fuzzing-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/fuzzing-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/fuzzing-2.csv'
]

interval_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/interval-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/interval-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/interval-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/interval-2.csv'
]

speed_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-2.csv'
]

systematic_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/systematic-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/systematic-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/systematic-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/systematic-2.csv'
]

rpmaccessory_files = [
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-accessory-2.csv'
]

speedaccessory_files = [
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-accessory-2.csv'
]

In [13]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in double_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Double Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Double Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 9.51 seconds

-----------------------------------------------------------------------
Accuracy: 99.61%
Precision: 99.26%
Recall: 99.61%
F1-Score: 99.43%
Matthews Correlation Coefficient: -0.0009
Informedness: 0.9921
Markedness: 0.9851


In [14]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in triple_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Triple Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Triple Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 8.40 seconds

-----------------------------------------------------------------------
Accuracy: 98.78%
Precision: 97.60%
Recall: 98.78%
F1-Score: 98.18%
Matthews Correlation Coefficient: -0.0015
Informedness: 0.9755
Markedness: 0.9520


In [15]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in fuzzing_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Fuzzing Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Fuzzing Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 7.43 seconds

-----------------------------------------------------------------------
Accuracy: 98.08%
Precision: 98.05%
Recall: 98.08%
F1-Score: 97.42%
Matthews Correlation Coefficient: 0.4131
Informedness: 0.9615
Markedness: 0.9610


In [16]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in interval_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Interval Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Interval Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 8.08 seconds

-----------------------------------------------------------------------
Accuracy: 98.18%
Precision: 96.43%
Recall: 98.18%
F1-Score: 97.30%
Matthews Correlation Coefficient: -0.0018
Informedness: 0.9636
Markedness: 0.9286


In [17]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in speed_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Speed Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Speed Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 13.89 seconds

-----------------------------------------------------------------------
Accuracy: 99.80%
Precision: 99.61%
Recall: 99.80%
F1-Score: 99.71%
Matthews Correlation Coefficient: -0.0004
Informedness: 0.9959
Markedness: 0.9923


In [18]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in systematic_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Systematic Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Systematic Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 5.68 seconds

-----------------------------------------------------------------------
Accuracy: 99.31%
Precision: 99.22%
Recall: 99.31%
F1-Score: 99.08%
Matthews Correlation Coefficient: 0.3670
Informedness: 0.9862
Markedness: 0.9843


In [19]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in rpmaccessory_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (RPM Accessory Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (RPM Accessory Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 1.03 seconds

-----------------------------------------------------------------------
Accuracy: 99.78%
Precision: 99.58%
Recall: 99.78%
F1-Score: 99.68%
Matthews Correlation Coefficient: -0.0005
Informedness: 0.9956
Markedness: 0.9917


In [20]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in speedaccessory_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Benchmark testing the BIRCH Model
test_metrics = test_birch(X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Speed Accessory Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Speed Accessory Attack Test Files):

-----------------------------------------------------------------------

Training Time: 1145.87 seconds
Testing Time: 1.14 seconds

-----------------------------------------------------------------------
Accuracy: 99.63%
Precision: 99.28%
Recall: 99.63%
F1-Score: 99.46%
Matthews Correlation Coefficient: -0.0006
Informedness: 0.9926
Markedness: 0.9857
